# Ch02. 텍스트 전처리 (Text Preprocessing)

# v06. 정수 인코딩 (Integer Encoding)

**텍스트를 숫자로 바꾸는 기법을 적용하기 위한 첫 단계**

- 각 단어를 고유한 정수에 맵핑(mapping)시키는 전처리 작업
- ex) 가지고 있는 텍스트에 5,000개의 단어가 있다.
  - 5,000개의 단어들 각각에 1번부터 5,000번까지 단어와 맵핑되는 **고유한 정수(인덱스)**를 부여
    - book -> 150번
    - dog -> 171번
    - love -> 192번
    - books -> 212번

**인덱스를 부여하는 방법**

- 랜덤으로 부여
- 전처리 또는 빈도수가 높은 단어들만 사용하기 위해서 단어에 대한 빈도수를 기준으로 정렬한 뒤에 부여

<br>

## 6.1 정수 인코딩 (Integer Encoding)

- 어떤 과정으로 단어에 정수 인덱스를 부여하는 지 확인

**단어의 정수를 부여하는 방법**

- 단어를 빈도수 순으로 정렬한 단어 집합(vocabulary)을 만듬
- 빈도수가 높은 순서대로 차례로 낮은 숫자부터 정수를 부여

- 단어의 빈도수가 적당하게 분포되도록 의도적으로 만든 텍스트 데이터를 가지고 실습

<br>

### 6.1.1 dictionary 사용하기

In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
text="A barber is a person. a barber is good person. a barber is huge person. \
he Knew A Secret! The Secret He Kept is huge secret. Huge secret. His barber kept his word. \
a barber kept his word. His barber kept his secret. \
But keeping and keeping such a huge secret to himself was driving the barber crazy. \
the barber went up a huge mountain."

<br>

#### 6.1.1.1 문장 토큰화

In [4]:
# 문장 토큰화
text = sent_tokenize(text)
print(text)

['A barber is a person.', 'a barber is good person.', 'a barber is huge person.', 'he Knew A Secret!', 'The Secret He Kept is huge secret.', 'Huge secret.', 'His barber kept his word.', 'a barber kept his word.', 'His barber kept his secret.', 'But keeping and keeping such a huge secret to himself was driving the barber crazy.', 'the barber went up a huge mountain.']


<br>

#### 6.1.1.2 정제 및 단어 토큰화

- 정제 작업을 병행하며 단어 토큰화 수행

In [ ]:
# 정제와 단어 토큰화
vocab = {} # 파이썬의 dictionary 자료형
sentences = []
stop_words = set(stopwords.words('english'))

for i in text:
  sentence = word_tokenize(i) # 단어 토큰화 수행
  result = []

  for word in sentence:
    word = word.lower() # 모든 단어를 소문자화하여 단어의 개수를 줄임
    if word not in stop_words: # 단어 토큰화된 결과에 대해서 불용어 제거
      if len(word) > 2: # 단어 길이가 2이하인 경우에 대해 단어 제거
        result.append(word)
        if word not in vocab:
          vocab[word] = 0
        vocab[word] += 1
  sentences.append(result)

In [6]:
print(sentences)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


<br>


#### 6.1.1.3 단어별 빈도수 확인

In [7]:
# 중복을 제거한 단어와 각 단어에 대한 빈도수
print(vocab)

{'barber': 8, 'person': 3, 'good': 1, 'huge': 5, 'knew': 1, 'secret': 6, 'kept': 4, 'word': 2, 'keeping': 2, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1}


- 단어를 키(key)로, 단어에 대한 빈도수가 값(value)으로 저장되어 있음

In [8]:
print(vocab["barber"]) # "barber" 라는 단어의 빈도수 출력

8


<br>

#### 6.1.1.4 빈도수가 높은 순서대로 정렬

In [9]:
vocab_sorted = sorted(vocab.items(), key=lambda x: x[1], reverse=True)
print(vocab_sorted)

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3), ('word', 2), ('keeping', 2), ('good', 1), ('knew', 1), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)]


<br>

#### 6.1.1.5 높은 빈도수를 가진 단어 순서대로 정수 인덱스 부여

In [ ]:
word_to_index = {}
i = 0

for (word, frequency) in vocab_sorted:
  if frequency > 1: # 정제(Cleaning) 챕터에서 언급했듯이 빈도수가 적은 단어 제외
    i = i+1
    word_to_index[word] = i  

In [11]:
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7}


<br>

#### 6.1.1.6 빈도수 상위 n개 단어 사용

- 텍스트 데이터에 있는 단어를 모두 사용하기 보다는 빈도수가 가장 높은 n개의 단어만 사용하고 싶은 경우가 많다.
- 위 단어들은 빈도수가 높은 순으로 낮은 정수가 부여되어 있음
- 빈도수 상위 n개의 단어만 사용하고 싶다.  
$\rightarrow$ `vocab`에서 정수값이 1부터 n까지인 단어들만 사용

- 상위 5개 단어만 사용

In [ ]:
vocab_size = 5

words_frequency = [w for w,c in word_to_index.items() if c >= vocab_size + 1] # 인덱스가 5 초과인 단어 제거

for w in words_frequency:
  del word_to_index[w] # 해당 단어에 대한 인덱스 정보 삭제

In [13]:
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


- `word_to_index`에는 빈도수가 높은 상위 5개의 단어만 저장됨

<br>

#### 6.1.1.7 단어를 정수로 인코딩

- `word_to_index`를 사용하여 단어 토큰화가 된 상태로 저장된 `sentences`에 있는 각 단어를 정수로 바꾼다.

In [15]:
print(sentences[0])

['barber', 'person']


- 첫 번째 문장인 `sentences[0]`에 대해 `[1,5]`로 인코딩한다.

In [16]:
print(sentences[1])

['barber', 'good', 'person']


- 그런데 두 번째 문장인 `sentences[1]`에는 `word_to_index`에는 존재하지 않는 단어인 'good'가 있다.

**Out-Of-Vocabulary(OOV)**

- 위에 처럼 훈련 데이터 또는 테스트 데이터에 대해서 단어 집합에 존재하지 않는 단어들을 OOV라고 한다.
- Out-Of-Vocabulary : 단어 집합에 없는 단어
- `word_to_index`에 `OOV`란 단어를 새롭게 추가
- 단어 집합에 없는 단어들은 `OOV`의 인덱스로 코딩한다.

In [ ]:
word_to_index['OOV'] = len(word_to_index) + 1

In [18]:
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'OOV': 6}


<br>

- 이제 `word_to_index`를 사용하여 `sentences`의 모든 단어들을 맵핑되는 정수로 인코딩

In [ ]:
encoded = []

for s in sentences:
  temp = []

  for w in s:
    try:
      temp.append(word_to_index[w])
    except KeyError:
      temp.append(word_to_index['OOV'])

  encoded.append(temp)

In [22]:
print(encoded)

[[1, 5], [1, 6, 5], [1, 3, 5], [6, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [6, 6, 3, 2, 6, 1, 6], [1, 6, 3, 6]]


<br>

- 정수 인코딩은 `Counter`, `FreqDist`, `enumerate` 또는 케라스 토크나이저를 사용하는 것을 권장한다.

<br>

### 6.1.2 `Counter` 사용하기

In [ ]:
from collections import Counter

In [24]:
print(sentences)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


- `sentences`는 단어 토큰화가 된 결과가 저장되어 있다.

<br>

#### 6.1.2.1 단어 집합 만들기

- 단어 집합(vocabulary)을 만들기 위해서 `sentences`에서 문장의 경계인 `[,]`를 제거하고 단어들을 하나의 리스트로 만든다.

In [25]:
words = sum(sentences, [])
# 위 작업은 words = np.hstack(sentences)로도 수행 가능
print(words)

['barber', 'person', 'barber', 'good', 'person', 'barber', 'huge', 'person', 'knew', 'secret', 'secret', 'kept', 'huge', 'secret', 'huge', 'secret', 'barber', 'kept', 'word', 'barber', 'kept', 'word', 'barber', 'kept', 'secret', 'keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy', 'barber', 'went', 'huge', 'mountain']


<br>

#### 6.1.2.2 `Counter()` : 중복 단어 제거 및 단어 빈도수 기록

- 파이썬의 `Counter()`의 입력으로 사용하면 중복을 제거하고 단어의 빈도수를 기록한다.

In [28]:
# 파이썬의 Counter 모듈을 이용하면 모든 빈도를 쉽게 계산할 수 있다.
vocab = Counter(words)
print(vocab)

Counter({'barber': 8, 'secret': 6, 'huge': 5, 'kept': 4, 'person': 3, 'word': 2, 'keeping': 2, 'good': 1, 'knew': 1, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1})


- 단어를 키(key)로, 단어에 대한 빈도수가 값(value)으로 저장되어 있다.

- `vocab`에 단어를 입력하면 빈도수를 리턴한다.

In [29]:
vocab['barber']

8

<br>

#### 6.1.2.3 `Counter().most_common()` : 상위 빈도수 단어 추출

- `most_common()`는 상위 빈도수를 가진 주어진 수의 단어만을 리턴한다.

- 등장 빈도수 상위 5개의 단어만 단어 집합으로 저장

In [30]:
vocab_size = 5
vocab = vocab.most_common(vocab_size)
vocab

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]

<br>

#### 6.1.2.4 정수 인덱스 인코딩

- 높은 빈도수를 가진 단어일수록 낮은 정수 인덱스를 부여

In [ ]:
word_to_index = {}
i = 0

for (word, frequency) in vocab:
  i = i+1
  word_to_index[word] = i

In [32]:
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


<br>

### 6.1.3 `NLTK`의 `FreqDist` 사용하기

- `NLTK`에서는 빈도수 계산 도구인 `FreqDist()`를 지원
- `Counter()`랑 같은 방법으로 사용할 수 있다.

In [ ]:
from nltk import FreqDist
import numpy as np

In [35]:
sentences

[['barber', 'person'],
 ['barber', 'good', 'person'],
 ['barber', 'huge', 'person'],
 ['knew', 'secret'],
 ['secret', 'kept', 'huge', 'secret'],
 ['huge', 'secret'],
 ['barber', 'kept', 'word'],
 ['barber', 'kept', 'word'],
 ['barber', 'kept', 'secret'],
 ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'],
 ['barber', 'went', 'huge', 'mountain']]

<br>

#### 6.1.3.1 단어 집합 생성

In [36]:
np.hstack(sentences)

array(['barber', 'person', 'barber', 'good', 'person', 'barber', 'huge',
       'person', 'knew', 'secret', 'secret', 'kept', 'huge', 'secret',
       'huge', 'secret', 'barber', 'kept', 'word', 'barber', 'kept',
       'word', 'barber', 'kept', 'secret', 'keeping', 'keeping', 'huge',
       'secret', 'driving', 'barber', 'crazy', 'barber', 'went', 'huge',
       'mountain'], dtype='<U8')

In [ ]:
# np.hstack으로 문장 구분을 제거하여 입력으로 사용
# ex) ['barber', 'person', 'barber', 'good' ... 중략 ...
vocab = FreqDist(np.hstack(sentences))

In [39]:
vocab

FreqDist({'barber': 8,
          'crazy': 1,
          'driving': 1,
          'good': 1,
          'huge': 5,
          'keeping': 2,
          'kept': 4,
          'knew': 1,
          'mountain': 1,
          'person': 3,
          'secret': 6,
          'went': 1,
          'word': 2})

In [41]:
print(vocab['barber'])

8


<br>

#### 6.1.3.2 `FreqDist().most_common()`

- `most_common()`은 상위 빈도수를 가진 주어진 수의 단어만을 리턴

In [42]:
vocab_size = 5
vocab = vocab.most_common(vocab_size)
vocab

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]

- `Counter()`의 `most_common()`와 결과가 동일

<br>

### 6.1.3.3 정수 인덱스 인코딩

- `enumerate()`를 사용하여 좀 더 짧은 코드로 인덱스를 부여

In [43]:
word_to_index = {word[0]: index + 1 for index, word in enumerate(vocab)}
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


<br>

### 6.1.4 `enumerate` 이해하기

- `enumerate()`는 순서가 있는 자료형(list, set, tuple, dictionary, string)을 입력으로 받아 인덱스를 순차적으로 함께 리턴한다.

In [44]:
test=['a', 'b', 'c', 'd', 'e']

for index, value in enumerate(test): # 입력의 순서대로 0부터 인덱스를 부여
  print('value : {}, index : {}'.format(value, index))


value : a, index : 0
value : b, index : 1
value : c, index : 2
value : d, index : 3
value : e, index : 4


<br>

## 6.2 케라스(Keras)의 텍스트 전처리

- 케라스(Keras)는 기본적인 전처리를 위한 도구를 제공

<br>

### 6.2.1 `Tokenizer`

- 정수 인코딩을 위해서 케라스의 전처리 도구인 토크나이저(`Tokenizer`)를 사용

In [45]:
%tensorflow_version 2.x

import tensorflow as tf
tf.__version__

TensorFlow 2.x selected.


'2.1.0'

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
sentences=[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], 
           ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], 
           ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], 
           ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], 
           ['barber', 'went', 'huge', 'mountain']]

<br>

#### 6.2.1.1 `fit_on_text()`

- `fit_on_texts()`는 입력한 텍스트로부터 단어 빈도수가 높은 순으로 낮은 정수 인덱스를 부여한다.
- 정수 인코딩 작업이 이루어진다고 보면 된다.
- `fit_on_texts()` 안에 코퍼스를 입력으로 하면 빈도수를 기준으로 단어 집합을 생성한다.

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)

<br>

#### 6.2.1.2 `word_index`

- 각 단어에 인덱스가 어떻게 부여되었는 지를 보려면, `word_index`를 사용한다.

In [49]:
print(tokenizer.word_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7, 'good': 8, 'knew': 9, 'driving': 10, 'crazy': 11, 'went': 12, 'mountain': 13}


- 각 단어의 빈도수가 높은 순서대로 인덱스가 부여된 것 확인

<br>

#### 6.2.1.3 `word_counts`

- 각 단어가 카운트를 수행했을 때 몇 개였는 지 확인

In [50]:
print(tokenizer.word_counts)

OrderedDict([('barber', 8), ('person', 3), ('good', 1), ('huge', 5), ('knew', 1), ('secret', 6), ('kept', 4), ('word', 2), ('keeping', 2), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)])


<br>

#### 6.2.1.4 `texts_to_sequences()`

- 입력으로 들어온 코퍼스에 대해서 각 단어를 이미 정해진 인덱스로 변환한다.

In [51]:
print(tokenizer.texts_to_sequences(sentences))

[[1, 5], [1, 8, 5], [1, 3, 5], [9, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [7, 7, 3, 2, 10, 1, 11], [1, 12, 3, 13]]


<br>

#### 6.2.1.5 `Tokenizer(num_words)`

- 위의 예제들에서 빈도수가 가장 높은 단어 n개만 사용하기 위해서 `most_common()`을 사용했다.
- 케라스 토크나이저에서는 다음과 같이 `Tokenizer()` 객체 생성 시`num_words` 매개변수를 지정하여  빈도수가 높은 상위 몇 개의 단어만 사용하겠다고 지정할 수 있다.

```python
tokenizer = Tokenizer(num_words=숫자)
```

- 상위 5개 단어를 사용한다고 토크나이저를 재정의

In [ ]:
vocab_size = 5
tokenizer = Tokenizer(num_words=vocab_size+1)
tokenizer.fit_on_texts(sentences)

**`num_words`에서 `+1`을 더해주는 이유**

- `num_words`는 숫자를 0부터 카운트한다.
- 만약 5를 넣으면 0 ~ 4번 단어 보존을 의미하게 된다.
- 따라서 1번 단어부터 4번 단어만 남게 된다.
- 그렇기 때문에 1 ~ 5번 단어까지 사용하고 싶다면 `num_words`에 `5 + 1`인 값을 넣어줘야 한다.

**케라스 토크나이저가 숫자 0까지 단어 집합의 크기로 산정하는 이유**

- 자연어 처리에서 **패딩(padding)**이라는 작업 때문이다.
- 추후에 학습할 내용이므로 우선 케라스 토크나이저를 사용할 때는 숫자 0도 단어 집합의 크기로 고려해야한다고 이해하고 넘어간다.

<br>

#### 6.2.1.6 `Tokenizer(num_words).word_index`

In [54]:
print(tokenizer.word_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7, 'good': 8, 'knew': 9, 'driving': 10, 'crazy': 11, 'went': 12, 'mountain': 13}


- 토크나이저 객체 생성 시 상위 5개의 단어만 사용하겠다고 선언하였는 데 여전히 13개의 단어가 모두 출력된다.

<br>

#### 6.2.1.7 `Tokenizer(num_words).word_counts`

In [55]:
print(tokenizer.word_counts)

OrderedDict([('barber', 8), ('person', 3), ('good', 1), ('huge', 5), ('knew', 1), ('secret', 6), ('kept', 4), ('word', 2), ('keeping', 2), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)])


- `word_counts`도 마찬가지로 13개의 단어가 모두 출력된다.

<br>

#### 6.2.1.8 `Tokenizer(num_words).texts_to_sequences()`

- 토크나이저 객체에 `num_words` 파라미터를 지정한 부분의 실제 적용은 `texts_to_sequences()`를 사용할 때 적용된다.

In [56]:
print(tokenizer.texts_to_sequences(sentences))

[[1, 5], [1, 5], [1, 3, 5], [2], [2, 4, 3, 2], [3, 2], [1, 4], [1, 4], [1, 4, 2], [3, 2, 1], [1, 3]]


- 코퍼스에 대해서는 각 단어를 이미 정해진 인덱스로 변환하는 데, 상위 5개의 단어만을 사용하겠다고 지정하였으므로 1번 단어부터 5번 단어까지만 보존되고 나머지 단어들을 제거된 것을 볼 수 있다.

- `word_index`와 `word_counts`에서도 지정된 `num_words` 만큼의 단어만 남기고 싶다면 아래의 코드로 구현할 수 있다.

In [58]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)

vocab_size = 5
words_frequency = [w for w, c in tokenizer.word_index.items() if c >= vocab_size + 1] # 인덱스가 5 초과인 단어 제거

for w in words_frequency:
  del tokenizer.word_index[w] # 해당 단어에 대한 인덱스 정보 삭제
  del tokenizer.word_counts[w] # 해당 단어에 대한 카운트 정보 삭제

print(tokenizer.word_index)
print(tokenizer.word_counts)
print(tokenizer.texts_to_sequences(sentences))

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}
OrderedDict([('barber', 8), ('person', 3), ('huge', 5), ('secret', 6), ('kept', 4)])
[[1, 5], [1, 5], [1, 3, 5], [2], [2, 4, 3, 2], [3, 2], [1, 4], [1, 4], [1, 4, 2], [3, 2, 1], [1, 3]]


<br>

#### 6.2.1.9 `Tokenizer(oov_token)`

- 케라스 토크나이저는 기본적으로 단어 집합에 없는 단어인 OOV에 대해서는 단어를 정수로 바꾸는 과정에서 아예 단어를 제거한다는 특징이 있다.
- 단어 집합에 없는 단어들은 OOV로 간주하여 보존하고 싶다면 `Tokenizer` 객체의 인자 `oov_token`을 사용한다.

In [ ]:
vocab_size = 5

tokenizer = Tokenizer(num_words=vocab_size+2, oov_token='OOV')
# 빈도수 상위 5개 단어만 사용. 숫자 0과 OOV를 고려해서 단어 집합의 크기는 +2
tokenizer.fit_on_texts(sentences)

- 만약 `oov_token`을 사용하기로 했다면 케라스 토크나이저는 기본적으로 'OOV'의 인덱스를 `1`로 한다.

In [60]:
print('단어 OOV의 인덱스 : {}'.format(tokenizer.word_index['OOV']))

단어 OOV의 인덱스 : 1


- 이제 코퍼스에 대해서 정수 인코딩을 진행

In [62]:
print(tokenizer.texts_to_sequences(sentences))

[[2, 6], [2, 1, 6], [2, 4, 6], [1, 3], [3, 5, 4, 3], [4, 3], [2, 5, 1], [2, 5, 1], [2, 5, 3], [1, 1, 4, 3, 1, 2, 1], [2, 1, 4, 1]]


- 빈도수 상위 5개의 단어는 2 ~ 6까지의 인덱스를 가짐
- 그 외 단어 집합에 없는 'goog'과 같은 단어들은 전부 'OOV'의 인덱스인 1로 인코딩됨